In [ ]:
from PIL import Image
import requests
from io import BytesIO


# pictures are in Database 
image_urls = [
    "https://upload.wikimedia.org/wikipedia/commons/e/e8/The_Joker_at_Wax_Museum_Plus.jpg", # Joker image
    "https://upload.wikimedia.org/wikipedia/en/9/98/Joker_%28DC_Comics_character%29.jpg" # Joker image
]

images = []
for url in image_urls:
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36" 
    }
    try:
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()
        image = Image.open(BytesIO(response.content)).convert("RGB")
        images.append(image)
    except requests.exceptions.RequestException as e:
        print(f"Failed to download {url}: {e}")

In [ ]:
from smolagents import CodeAgent, OpenAIServerModel
import os

## Musim si zaplatit za API key 
os.environ["OPENAI_API_KEY"] = 

api_key = os.environ.get("OPENAI_API_KEY")
if not api_key:
    raise ValueError("Please set your OpenAI API key in the OPENAI_API_KEY environment variable.")

model = OpenAIServerModel(model_id="gpt-4o", api_key=api_key)

# Instantiate the agent
agent = CodeAgent(
    tools=[],
    model=model,
    max_steps=20,
    verbosity_level=2
)

response = agent.run(
    """
    Describe the costume and makeup that the comic character in these photos is wearing and return the description.
    Tell me if the guest is The Joker or Wonder Woman.
    """,
    images=images
)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Describe the costume and makeup that the comic character in these photos is wearing and return the description. │
│     Tell me if the guest is The Joker or Wonder Woman.                                                          │
│                                                                                                                 │
╰─ OpenAIServerModel - gpt-4o ────────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Error in generating model output:
Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing 
details. For more information on this error, read the docs: 
https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 
'code': 'insufficient_quota'}}

[Step 1: Duration 45.63 seconds]

AgentGenerationError: Error in generating model output:
Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

### We will use `Dynamic` Retrieval for Pictures that are not in database

- we will retrieve images from external sources 

MultiStepAgent = ReAct Framework

#### This class operates in a structured cycle where various variables and knowledge are logged at different stages:

`SystemPromptStep`: Stores the system prompt.

`TaskStep`: Logs the user query and any provided input.

`ActionStep`: Captures logs from the agent’s actions and results.

When browsing, the agent can take screenshots and save them as observation_images in the ActionStep.

In [15]:
from smolagents import tool
from selenium import webdriver
from selenium.webdriver.common.by import By

# Initialize the Selenium WebDriver (Chrome in this example)
driver = webdriver.Chrome()


@tool
def search__item_ctrl_f(text: str, nth_result: int = 1) -> str:
    """
    Searches for text on the current page via Ctrl+F and returns the nth result.

    Args:
        text: The text to search for
        nth_result: Which occurence of the text to return (default is 1, which returns the first occurrence)
    """

    elements = driver.find_elements(By.XPATH, f"//*[contains(text(),'{text}')]")
    if nth_result <= len(elements):
        raise Exception(f"Match n°{nth_result} not found (only {len(elements)} matches found)")
    
    result=f"Found {len(elements)} matches for '{text}'. "
    elem= elements[nth_result - 1]
    driver.execute_script("arguments[0].scrollIntoView();", elem)
    result += f"Focused on element: {nth_result} with text: {len(elements)}"
    return result



@tool
def go_back() -> None:
    """
    Goes back to the previous page in the browser history.
    """
    driver.back()
    return "Went back to the previous page."


@tool
def close_popups() -> str:
    """
    Closes the current popup window if it exists.
    """
    webdriver.ActionChains(driver).send_keys(webdriver.Keys.ESCAPE).perform()

Error sending stats to Plausible: error sending request for url (https://plausible.io/api/event)
